In [31]:
import bs4
import traceback
import re
import time
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome import service as fs
 
# ドライバーのフルパス
CHROMEDRIVER = "C:/****/****/****/chromedriver_win32/chromedriver.exe"
# 改ページ（最大）
PAGE_MAX = 12
# 遷移間隔（秒）
INTERVAL_TIME = 3
# 対象年度
YEAR = [2018, 2019, 2020, 2021]
 

# ドライバー準備
def get_driver():
    # ヘッドレスモードでブラウザを起動
    options = Options()
    options.add_argument('--headless')
 
    # ブラウザーを起動
    # ドライバー指定でChromeブラウザを開く 
    chrome_service = fs.Service(executable_path=CHROMEDRIVER) 
    driver = webdriver.Chrome(service=chrome_service, options=options)
    
    return driver
 
 
# 対象ページのソース取得
def get_source_from_page(driver, page):
    try:
        # ターゲット
        driver.get(page)
        driver.implicitly_wait(10)  # 見つからないときは、10秒まで待つ
        page_source = driver.page_source
 
        return page_source
 
    except Exception as e:
 
        print("Exception\n" + traceback.format_exc())
 
        return None
 
 
# ソースからスクレイピングする
def get_data_from_source(src):
    # スクレイピングする
    soup = bs4.BeautifulSoup(src, features='lxml')
 
    try:
        info = []
        table = soup.find(class_="Calendar_Table")
 
        if table:
            elems = table.find_all("td", class_="RaceCellBox")
 
            for elem in elems:
                a_tag = elem.find("a")
 
                if a_tag:
                    href = a_tag.attrs['href']
                    match = re.findall("\/top\/race_list.html\?kaisai_date=(.*)$", href)
 
                    if len(match) > 0:
                        item_id = match[0]
                        info.append(item_id)
 
        return info
 
    except Exception as e:
 
        print("Exception\n" + traceback.format_exc())
 
        return None
 
 
if __name__ == "__main__":
 
    # ブラウザのdriver取得
    driver = get_driver()
 
    # ページカウンター制御
    page_counter = 0
    
    datalist = []
    
    for year in YEAR:
        for month in range(1, 13):
 
            page_counter = page_counter + 1
 
            # 対象ページURL
            page = "https://race.netkeiba.com/top/calendar.html?year=" + str(year) + "&month=" + str(month)
 
            # ページのソース取得
            source = get_source_from_page(driver, page)
 
            # ソースからデータ抽出
            data = get_data_from_source(source)
            datalist.extend(data)
 
            # 間隔を設ける(秒単位）
            time.sleep(INTERVAL_TIME)
 
            # 改ページ処理を抜ける
            if page_counter == PAGE_MAX:
                break
 
        # 改ページ処理を抜ける
        if page_counter == PAGE_MAX:
            break
    # 閉じる
    driver.quit()
    
    # データ保存
    file = open('C:/Users/jouza/Desktop/racefile/racedate.bin', 'wb')
    pickle.dump(datalist,file)
    file.close
    print(datalist)

['20180106', '20180107', '20180108', '20180113', '20180114', '20180120', '20180121', '20180127', '20180128', '20180203', '20180204', '20180210', '20180211', '20180212', '20180213', '20180217', '20180218', '20180224', '20180225', '20180303', '20180304', '20180310', '20180311', '20180317', '20180318', '20180324', '20180325', '20180331', '20180401', '20180407', '20180408', '20180414', '20180415', '20180421', '20180422', '20180428', '20180429', '20180505', '20180506', '20180512', '20180513', '20180519', '20180520', '20180526', '20180527', '20180602', '20180603', '20180609', '20180610', '20180616', '20180617', '20180623', '20180624', '20180630', '20180701', '20180707', '20180708', '20180714', '20180715', '20180721', '20180722', '20180728', '20180729', '20180804', '20180805', '20180811', '20180812', '20180818', '20180819', '20180825', '20180826', '20180901', '20180902', '20180908', '20180909', '20180915', '20180916', '20180917', '20180922', '20180923', '20180929', '20180930', '20181002', '20

In [ ]:
import bs4
import traceback
import re
import time
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome import service as fs
 
# ドライバーのフルパス
CHROMEDRIVER = "C:/Users/jouza/Downloads/chromedriver_win32/chromedriver.exe"
# 改ページ（最大）
PAGE_MAX = 120
# 遷移間隔（秒）
INTERVAL_TIME = 3
 
 
# ドライバー準備
def get_driver():
    # ヘッドレスモードでブラウザを起動
    options = Options()
    options.add_argument('--headless')
 
    # ブラウザーを起動
    chrome_service = fs.Service(executable_path=CHROMEDRIVER) 
    driver = webdriver.Chrome(service=chrome_service, options=options)
 
    return driver
 
 
# 対象ページのソース取得
def get_source_from_page(driver, page):
    try:
        # ターゲット
        driver.get(page)
        # id="RaceTopRace"の要素が見つかるまで10秒は待つ
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'RaceTopRace')))
        page_source = driver.page_source
 
        return page_source
 
    except Exception as e:
 
        print("Exception\n" + traceback.format_exc())
 
        return None
 
 
# ソースからスクレイピングする
def get_data_from_source(src):
    # スクレイピングする
    soup = bs4.BeautifulSoup(src, features='lxml')
 
    try:
        info = []
        elem_base = soup.find(id="RaceTopRace")
 
        if elem_base:
            elems = elem_base.find_all("li", class_="RaceList_DataItem")
 
            for elem in elems:
                # 最初のaタグ
                a_tag = elem.find("a")
 
                if a_tag:
                    href = a_tag.attrs['href']
                    match = re.findall("\/race\/result.html\?race_id=(.*)&rf=race_list", href)
                    if len(match) > 0:
                        item_id = match[0]
                        info.append(item_id)
        return info
 
    except Exception as e:
 
        print("Exception\n" + traceback.format_exc())
 
        return None
 
# kaisai_dateリストを取得する
def get_list_id():
    f = open('C:/Users/jouza/Desktop/racefile/racedate.bin', 'rb')
    race_list_id = pickle.load(f)
    # print(race_list_id)
    return race_list_id
 
 
if __name__ == "__main__":
 
    datalist = []
    
    # kaisai_dateリスト取得
    list_id = get_list_id()
 
    # ブラウザのdriver取得
    driver = get_driver()
 
    # ページカウンター制御
    page_counter = 0
 
    for kaisai_date in list_id:
 
        page_counter = page_counter + 1
 
        # 対象ページURL
        page = "https://race.netkeiba.com/top/race_list.html?kaisai_date=" + str(kaisai_date)
 
        # ページのソース取得
        source = get_source_from_page(driver, page)
        # print(source)
 
        # ソースからデータ抽出
        data = get_data_from_source(source)
        print(data)
        datalist.extend(data)
        
        # 間隔を設ける(秒単位）
        time.sleep(INTERVAL_TIME)
 
        # 改ページ処理を抜ける
        if page_counter == PAGE_MAX:
            break
 
 
    # 閉じる
    driver.quit()

# データ保存
file = open('C:/Users/jouza/Desktop/racefile/raceid.bin', 'wb')
pickle.dump(datalist,file)
file.close
print(datalist)

In [29]:
import bs4
import traceback
import re
import time
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome import service as fs

CHROMEDRIVER = "C:/Users/jouza/Downloads/chromedriver_win32/chromedriver.exe"
# 改ページ（最大）
PAGE_MAX = 2
# 遷移間隔（秒）
INTERVAL_TIME = 3
 
 
# ドライバー準備
def get_driver():
    # ヘッドレスモードでブラウザを起動
    options = Options()
    options.add_argument('--headless')
 
    # ブラウザーを起動
    chrome_service = fs.Service(executable_path=CHROMEDRIVER) 
    driver = webdriver.Chrome(service=chrome_service, options=options)
 
    return driver
 
 
# 対象ページのソース取得
def get_source_from_page(driver, page):
    try:
        # ターゲット
        driver.get(page)
        # class="RaceList_NameBox"の要素が見つかるまで10秒は待つ
        target_elem = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "RaceList_NameBox"))
        )
 
        if target_elem:
            page_source = driver.page_source
            return page_source
        else:
            return None
 
    except Exception as e:
 
        print("Exception\n" + traceback.format_exc())
 
        return None
 
 
# ソースからスクレイピングする
def get_data_from_source(src):
 
    try:
        # スクレイピングする
        soup = bs4.BeautifulSoup(src, features='lxml')
 
        info = {}
        info["race_info"] = None
        info["race_order"] = None
        info["payout"] = None
        info["rap_pace"] = None
 
        # レース情報取得
        info["race_info"] = get_race_info(soup)
        # 全着順取得
        info["race_order"] = get_order(soup)
        # 払い戻し取得
        info["payout"] = get_payout(soup)
        # ラップタイム取得
        info["rap_pace"] = get_rap_pace(soup)
 
        return info
 
    except Exception as e:
 
        print("Exception\n" + traceback.format_exc())
 
        return None
 
 
# レース情報の抽出
def get_race_info(soup):
 
    result = {}
    result["no"] = None
    result["name"] = None
    result["time"] = None
    result["kind"] = None
    result["weather"] = None
    result["state"] = None
    result["course"] = None
    result["etc_1"] = None
    result["etc_2"] = None
    result["etc_3"] = None
    result["etc_4"] = None
    result["etc_5"] = None
    result["etc_6"] = None
    result["etc_7"] = None
    result["etc_8"] = None
 
    elem_base = soup.find(class_="RaceList_NameBox")
    if elem_base:
        tmp_elem = elem_base.find(class_="RaceNum")
        if tmp_elem:
            tmp_data = tmp_elem.text
            result["no"] = my_trim(tmp_data)
 
        tmp_elem = elem_base.find(class_="RaceName")
        if tmp_elem:
            tmp_data = tmp_elem.text
            result["name"] = my_trim(tmp_data)
 
        tmp_elem = elem_base.find(class_="RaceData01")
        if tmp_elem:
            tmp_data = tmp_elem.text
            tmp_data_list = tmp_data.split("/")
            if len(tmp_data_list) >= 4:
                result["time"] = my_trim(tmp_data_list[0])
                result["kind"] = my_trim(tmp_data_list[1])
                result["weather"] = my_trim(tmp_data_list[2])
                result["state"] = my_trim(tmp_data_list[3])
 
        tmp_elem = elem_base.find(class_="RaceData02")
        if tmp_elem:
            elems = tmp_elem.find_all("span")
            if len(elems) >=9:
                result["course"] = my_trim(elems[1].text)
                result["etc_1"] = my_trim(elems[0].text)
                result["etc_2"] = my_trim(elems[2].text)
                result["etc_3"] = my_trim(elems[3].text)
                result["etc_4"] = my_trim(elems[4].text)
                result["etc_5"] = my_trim(elems[5].text)
                result["etc_6"] = my_trim(elems[6].text)
                result["etc_7"] = my_trim(elems[7].text)
                result["etc_8"] = my_trim(elems[8].text)
 
    return result
 
 
# 全着順の抽出
def get_order(soup):
 
    result = []
 
    elem_base = soup.find(id="All_Result_Table")
    if elem_base:
        tr_elems = elem_base.find_all("tr", class_="HorseList")
 
        for tr_elem in tr_elems:
            tmp = {}
            td_elems = tr_elem.find_all("td")
 
            if len(td_elems)==15:
                tmp["rank"] = my_trim(td_elems[0].text)
                tmp["waku"] = my_trim(td_elems[1].text)
                tmp["umaban"] = my_trim(td_elems[2].text)
                tmp["horse_name"] = my_trim(td_elems[3].text)
                tmp["horse_age"] = my_trim(td_elems[4].text)
                tmp["jockey_weight"] = my_trim(td_elems[5].text)
                tmp["jockey_name"] = my_trim(td_elems[6].text)
                tmp["time_1"] = my_trim(td_elems[7].text)
                tmp["time_2"] = my_trim(td_elems[8].text)
                tmp["odds_1"] = my_trim(td_elems[9].text)
                tmp["odds_2"] = my_trim(td_elems[10].text)
                tmp["time_3"] = my_trim(td_elems[11].text)
                tmp["passage_rate"] = my_trim(td_elems[12].text)
                tmp["trainer_name"] = my_trim(td_elems[13].text)
                tmp["horse_weight"] = my_trim(td_elems[14].text)
 
                # 馬ID
                a_tag = td_elems[3].find("a")
                if a_tag:
                    href = a_tag.attrs['href']
                    match = re.findall("\/horse\/(.*)$", href)
                    if len(match) > 0:
                        tmp_id = match[0]
                        tmp["horse_id"] = tmp_id
 
                # 騎手ID
                a_tag = td_elems[6].find("a")
                if a_tag:
                    href = a_tag.attrs['href']
                    match = re.findall("\/jockey\/(.*)\/", href)
                    if len(match) > 0:
                        tmp_id = match[0]
                        tmp["jockey_id"] = tmp_id
 
                # 厩舎ID
                a_tag = td_elems[13].find("a")
                if a_tag:
                    href = a_tag.attrs['href']
                    match = re.findall("\/trainer\/(.*)\/", href)
                    if len(match) > 0:
                        tmp_id = match[0]
                        tmp["trainer_id"] = tmp_id
 
            result.append(tmp)
 
    return result
 
 
# 払い戻し取得
def get_payout(soup):
 
    result = {}
 
    elem_base = soup.find(class_="FullWrap")
    if elem_base:
        tr_elems = elem_base.find_all("tr")
 
        for tr_elem in tr_elems:
 
            row_list = []
 
            class_name = tr_elem.attrs["class"]
            # class名を小文字にに変換
            class_name = class_name[0].lower()
 
            td_elems = tr_elem.find_all("td")
            if len(td_elems) == 3:
 
                # Ninkiのspan数が行数と判断可能
                span_elems = td_elems[2].find_all("span")
                count = len(span_elems)
                # Payoutのテキストをbrで分割してできるデータ数とcountが同じ
                # ただ、分割は「円」で行う
                payout_text = td_elems[1].text
                payout_text_list = payout_text.split("円")
 
                if class_name=="tansho" or class_name=="fukusho":
                    # Resultのdiv数がcountの3倍
                    target_elems = td_elems[0].find_all("div")
                else:
                    # Resultのul数がcountと同じ
                    target_elems = td_elems[0].find_all("ul")
 
                for i in range(count):
                    tmp = {}
                    tmp["payout"] = my_trim(payout_text_list[i]) + "円"
                    tmp["ninki"] = my_trim(span_elems[i].text)
 
                    target_str = ""
                    if class_name == "tansho" or class_name == "fukusho":
                        target_str = my_trim(target_elems[i*3].text)
                    else:
                        li_elems = target_elems[i].find_all("li")
                        for li_elem in li_elems:
                            tmp_str = my_trim(li_elem.text)
                            if tmp_str:
                                target_str = target_str + "-" + tmp_str
                        # 先頭の文字を削除
                        target_str = target_str.lstrip("-")
 
                    tmp["result"] = target_str
 
                    row_list.append(tmp)
 
            result[class_name] = row_list
 
    return result
 
# ラップタイム取得
def get_rap_pace(soup):
 
    result = []
 
    row_list = []
 
    elem_base = soup.find(class_="Race_HaronTime")
    if elem_base:
        tr_elems = elem_base.find_all("tr")
 
        counter = 0
        for tr_elem in tr_elems:
 
            col_list = []
            if  counter == 0:
                target_elems = tr_elem.find_all("th")
            else:
                target_elems = tr_elem.find_all("td")
 
            for target_elem in target_elems:
                tmp_str = my_trim(target_elem.text)
                col_list.append(tmp_str)
 
            row_list.append(col_list)
 
            counter = counter + 1
 
    for i in range(len(row_list[0])):
        tmp = {}
        tmp["header"] = row_list[0][i]
        tmp["haron_time_1"] = row_list[1][i]
        tmp["haron_time_2"] = row_list[2][i]
 
        result.append(tmp)
 
    return result
 
 
# 数値だけ抽出
def extract_num(val):
    num = None
    if val:
        match = re.findall("\d+\.\d+", val)
        if len(match) > 0:
            num = match[0]
        else:
            num = re.sub("\\D", "", val)
 
    if not num:
        num = 0
 
    return num
 
 
def my_trim(text):
    text = text.replace("\n", "")
    return text.strip()
 
 
# race_idリスト取得
def get_list_id():
    f = open('C:/Users/jouza/Desktop/racefile/raceid.bin', 'rb')
    race_list_id = pickle.load(f)
    # print(race_list_id)
    return race_list_id
 
if __name__ == "__main__":
 
    # kaisai_dateリスト取得
    list_id = get_list_id()
 
    # ブラウザのdriver取得
    driver = get_driver()
 
    # ページカウンター制御
    page_counter = 0
 
    for race_id in list_id:
 
        page_counter = page_counter + 1
 
        # 対象ページURL
        page = "https://race.netkeiba.com/race/result.html?race_id=" + str(race_id)
 
        # ページのソース取得
        source = get_source_from_page(driver, page)
 
        # ソースからデータ抽出
        data = get_data_from_source(source)
 
        # データ保存
        # print(data)
 
        # 間隔を設ける(秒単位）
        time.sleep(INTERVAL_TIME)
 
        # 改ページ処理を抜ける
        if page_counter == PAGE_MAX:
            break
 
 
    # 閉じる
    driver.quit()

{'race_info': {'no': '1R', 'name': '3歳未勝利', 'time': '09:55発走', 'kind': 'ダ1200m (右)', 'weather': '天候:晴', 'state': '馬場:良', 'course': '中山', 'etc_1': '1回', 'etc_2': '1日目', 'etc_3': 'サラ系３歳', 'etc_4': '未勝利', 'etc_5': '[指]', 'etc_6': '馬齢', 'etc_7': '16頭', 'etc_8': '本賞金:500,200,130,75,50万円'}, 'race_order': [{'rank': '1', 'waku': '8', 'umaban': '15', 'horse_name': 'コウギョウブライト', 'horse_age': '牡3', 'jockey_weight': '56.0', 'jockey_name': '戸崎圭', 'time_1': '1:12.9', 'time_2': '', 'odds_1': '1', 'odds_2': '2.0', 'time_3': '38.7', 'passage_rate': '2-2', 'trainer_name': '美浦伊藤大', 'horse_weight': '462(0)', 'horse_id': '2015104603', 'jockey_id': 'result/recent/05386', 'trainer_id': 'result/recent/01109'}, {'rank': '2', 'waku': '2', 'umaban': '3', 'horse_name': 'ユメイチズ', 'horse_age': '牡3', 'jockey_weight': '56.0', 'jockey_name': '松岡', 'time_1': '1:13.0', 'time_2': '1/2', 'odds_1': '2', 'odds_2': '6.7', 'time_3': '37.8', 'passage_rate': '9-6', 'trainer_name': '美浦浅野', 'horse_weight': '478(+4)', 'horse_id': '2